# Price Discovery Project Using RAG

## Import Libraries

In [2]:
import os
import torch
import shutil
import tqdm
import open_clip
import qdrant_client
import pandas as pd
from sklearn.model_selection import train_test_split
from langchain_community.document_loaders import DirectoryLoader, CSVLoader
from langchain_community.document_loaders.image import UnstructuredImageLoader
from langchain_community.vectorstores import Chroma, Qdrant
from langchain_experimental.open_clip import OpenCLIPEmbeddings


In [3]:
# Specify variables
model_name = "ViT-B-16"
checkpoint = "openai"
embedding_model= OpenCLIPEmbeddings(model_name=model_name, checkpoint=checkpoint)
db_dir = "db"


## Load Data

In [7]:
# Read the text data

# Instatiate loader 
text_loader = CSVLoader("train.csv",
                        encoding="utf-8",
                        csv_args={
                            "delimiter": ",",
                            "quotechar": '"',
                        })

# Load the data
text_data = text_loader.load()

## Create Vector DB to store data

In [4]:
# Instantiate DB
text_db = Chroma(persist_directory=db_dir, embedding_function=embedding_model, collection_name="text-collection")
images_db = Chroma(persist_directory=db_dir, embedding_function=embedding_model, collection_name="images-collection")

In [57]:
text_db.add_documents(text_data)

['8187329a-d59a-11ee-9bee-c0b6f9d0c838',
 '81876dba-d59a-11ee-a5ff-c0b6f9d0c838',
 '81876dbb-d59a-11ee-bea2-c0b6f9d0c838',
 '81876dbc-d59a-11ee-9f02-c0b6f9d0c838',
 '81876dbd-d59a-11ee-91bd-c0b6f9d0c838',
 '81876dbe-d59a-11ee-ac1c-c0b6f9d0c838',
 '81876dbf-d59a-11ee-a066-c0b6f9d0c838',
 '81876dc0-d59a-11ee-84ec-c0b6f9d0c838',
 '81876dc1-d59a-11ee-b111-c0b6f9d0c838',
 '81876dc2-d59a-11ee-8a20-c0b6f9d0c838',
 '81876dc3-d59a-11ee-8b8b-c0b6f9d0c838',
 '81876dc4-d59a-11ee-b5fe-c0b6f9d0c838',
 '81876dc5-d59a-11ee-9d4b-c0b6f9d0c838',
 '81876dc6-d59a-11ee-9161-c0b6f9d0c838',
 '81876dc7-d59a-11ee-bcf0-c0b6f9d0c838',
 '81876dc8-d59a-11ee-8140-c0b6f9d0c838',
 '81876dc9-d59a-11ee-bfab-c0b6f9d0c838',
 '81876dca-d59a-11ee-9416-c0b6f9d0c838',
 '81876dcb-d59a-11ee-9a19-c0b6f9d0c838',
 '81876dcc-d59a-11ee-a69b-c0b6f9d0c838',
 '81876dcd-d59a-11ee-a8c2-c0b6f9d0c838',
 '81876dce-d59a-11ee-b228-c0b6f9d0c838',
 '81876dcf-d59a-11ee-9035-c0b6f9d0c838',
 '81876dd0-d59a-11ee-a4f4-c0b6f9d0c838',
 '81876dd1-d59a-

In [108]:
# Instantiate DB
text_test_db = Chroma(persist_directory="Databases/clip_textdb", embedding_function=embedding_model, collection_name="Text_Store")
images_test_db = Chroma(persist_directory="Databases/product_imagedb", embedding_function=embedding_model, collection_name="Image_Store")

## Create agent for retrieval

In [134]:
from langchain.tools.retriever import create_retriever_tool
from langchain.agents.load_tools import load_tools
from langchain.agents import AgentExecutor, Tool, create_structured_chat_agent, create_react_agent, create_self_ask_with_search_agent
from langchain_community.utilities import SearchApiAPIWrapper, SerpAPIWrapper
from langchain import hub
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain_community.tools.google_lens import GoogleLensQueryRun
from langchain_community.utilities.google_lens import GoogleLensAPIWrapper
from langchain_community.utilities import GoogleSearchAPIWrapper
from langchain.agents.agent import AgentAction
from langchain_core.callbacks.base import BaseCallbackHandler
from langchain_community.llms import HuggingFaceEndpoint
from typing import List, Dict, Any, Union

In [109]:
# Instantiate retrievers from db

text_retriever = text_db.as_retriever()


In [8]:
# Instantiate langsmith for visualizing metrivs
langchain_api_key = ""
os.environ["LANGCHAIN_API_KEY"] = langchain_api_key
os.environ["LANGCHAIN_PROJECT"] = "price-discovery"
os.environ["LANGCHAIN_TRACING_V2"] = "true"

In [138]:
HUGGINGFACEHUB_API_TOKEN = ""
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN

repo_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"

llm_two = HuggingFaceEndpoint(
    repo_id=repo_id
)



Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to C:\Users\Spectra\.cache\huggingface\token
Login successful


In [9]:
# Setup google api
GOOGLE_API_KEY = ""  
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

llm = ChatGoogleGenerativeAI(model="gemini-pro", google_api_key=GOOGLE_API_KEY, temperature=0.3)

In [10]:
# setup serp api
serp_api_key = ""
os.environ["SERPAPI_API_KEY"] = serp_api_key

In [124]:
# Create prompts
test_prompt = """
    
As an experienced product analyst adept at accurately determining product price ranges,utilize the available tools {tools} to answer the question asked. 
Return ONLY a reasonable, accurate and concise price range for my product after analysing its current pricing
and that of a similar products. 

    Use the following format:

    Question: the input question you must answer
    Thought: you should always think about what to do
    Action: the action to take, should be one of [{tool_names}]
    Action Input: the input to the action
    Observation: the result of the action
    ... (this Thought/Action/Action Input/Observation can repeat N times). If no result is found use your own knowledge
    Thought: I now know the final answer
    Final Answer: the final answer to the original input question

    Begin!

    Question: {input}
    Thought: {agent_scratchpad}
"""

test_prompt_one = """    
As an experienced product analyst adept at accurately determining product price ranges, utilize ALL the available tools {tools} to answer the question asked accurately. 
Check whether the product is in the database, then search the internet and compare the prices.
If a tool encounters an error, use another tool.
Return ONLY a reasonable and accurate  PRICE RANGE for the product.
Use the following format:

Question: the input question you must answer.
Thought: you should always think about what to do. 
Action: the action to take, should be ONE of or ALL of [{tool_names}]. 
Action Input: the input to the action. It SHOULD be a string.
Action: choose another tool if a tool fails or handle its error.
Observation: the result of the action, including insights gained or findings.
... (this Thought/Action/Action Input/Observation can repeat N times). If no result is found, use your own knowledge.
Thought: I now know the final answer that is based on the analysis of your findings and observations.
Final Answer: the final answer to the original input question, which must be as accurate as possible. NEVER say you can't answer.

Begin!

Question: {input}
Thought: {agent_scratchpad}

"""

test_prompt_two = """

As an experienced product analyst proficient in determining accurate product price ranges, utilize all available tools {tools}, including internet searches and databases,
to precisely answer the given question. Begin by checking the database, then proceed to search the internet.Then pick the most similar products and use them to come up with an accurate price range.
Similar products are those with closely matching specifications based on product information and brand. 
Based on this comparison, generate a highly accurate, reasonable, and compact estimate of the price range for the product
If a tool encounters an error, use another tool.
Return ONLY the ESTIMATE of the PRICE RANGE for the product. An example being $100-110 with the price of the product being $106

Use the following format:
Question: the input question you must answer.
Thought: you should always think about what to do. 
Action: the action to take, should be ONE of[{tool_names}]. 
Action Input: the input to the action. It SHOULD be a string.
Action: choose another tool if a tool fails or handle its error.
Observation: the result of the action, including insights gained or findings.
... (this Thought/Action/Action Input/Observation can repeat N times). If no result is found, use your own knowledge.
Thought: I now know the final answer that is based on the analysis of your findings and observations.
Final Answer: the final answer to the original input question, which must be as accurate as possible. NEVER say you can't answer.

An example to follow is:

Question: what is the estimated, accurate and compact price range of a Sony WH-CH520 Wireless Headphones Bluetooth On-Ear Headset with Microphone, Black New?
Thought 1: I need to search for the Sony WH-CH520 Wireless Headphones to identify their price range.
Action 1: search text db
Action Input 1: : Sony WH-CH520 Wireless Headphones.
Observation 1: The search results providing various listings for products similar to the Sony WH-CH520 Wireless Headphones with different prices.
Action 2: search internet 
Action Input 2: Sony WH-CH520 Wireless Headphones Bluetooth On-Ear Headset with Microphone.
Observation 2: The search results provide various listings for the Sony WH-CH520 Wireless Headphones with different prices.
Thought 3: I need to check the obtained products for similarity.
Action 3: The products with great similarity to the Sony WH-CH520 Wireless Headphones are chosen.
Thought 4: I need to gather information on the range of prices offered for these headphones.
Action 4: Compare and analyze the search results to determine the price range.
Observation 4: Prices for the Sony WH-CH520 Wireless Headphones range from $80 to $100 across different retailers and platforms, based on comparing to very similar products with similar specifications and brand.
Thought 5: The price range of the Sony WH-CH520 Wireless Headphones is between $80 and $100.
Final Answer: $80 - $100


Begin!

Question: {input}
Thought: {agent_scratchpad}
"""

test_prompt_three = """
As an experienced product analyst proficient in determining accurate product price ranges, utilize all available tools {tools}, including internet searches and databases,
to precisely answer the given question. Begin by checking the database, then proceed to search the internet.Then pick the most similar products and use them to come up with an accurate price range.
Similar products are those with closely matching specifications based on product information and brand. 
Based on this comparison, generate a highly accurate, reasonable, and compact estimate of the price range for the product
If a tool encounters an error, use another tool.
Return ONLY the ESTIMATE of the PRICE RANGE for the product. An example being $100-110 with the price of the product being $106
Use the following format:

Question: the input question you must answer.
Thought: you should always think about what to do. 
Action: the action to take, should be ONE of or ALL of [{tool_names}]. 
Action Input: the input to the action. It SHOULD be a string.
Action: choose another tool if a tool fails or handle its error.
Observation: the result of the action, including insights gained or findings.
... (this Thought/Action/Action Input/Observation can repeat N times). If no result is found, use your own knowledge.
Thought: I should recheck my answer and make sure it meets the specifications. If not, try again before coming up with the final answer.
Thought: I now know the final answer that is based on the indepth analysis of your findings and observations. 
Final Answer: the final answer to the original input question, which must be as accurate and compact as possible. NEVER say you can't answer.

Begin!

Question: {input}
Thought: {agent_scratchpad}

"""

In [128]:
# Create prompt template from prompts
test_prompt_template = PromptTemplate.from_template(test_prompt_three)

In [78]:
# Instantiate error handler
class RetryCallbackHandler(BaseCallbackHandler):
    def on_tool_error(
        self, error: Union[Exception, KeyboardInterrupt], **kwargs: Any
    ) -> Any:
        # Log the error or perform any necessary actions
        
        # Retry logic
        max_retries = 1
        current_retry = kwargs.get('retry_count', 0)
        if current_retry < max_retries:
            print(f"Retrying tool execution (Attempt {current_retry + 1})")
            # Increment retry count
            kwargs['retry_count'] = current_retry + 1
            # Re-run the tool
            return AgentAction.RERUN
        else:
            print("Maximum retries reached. Switching to another tool.")
            return AgentAction.CHANGE_TOOL

In [22]:
# The error handler
error_handler = RetryCallbackHandler()

In [55]:
# Create tools
search = SerpAPIWrapper()
tool_search_item = Tool(
    name="search internet",
    description="Searches the internet for the price of a product by using its description.",
    func=search.run,
    handle_tool_error=True
)

# Tool for searching product price using image URL
tool_search_image = Tool(
    name="search image",
    description="Searches for the price of a product using its image URL.",
    func=search.run,
    handle_tool_error=True
)

# Tool for retrieving product information from the text database based on the product description
tool_retrieve_from_text_db = create_retriever_tool(
    text_retriever,
    'search text db',
    'Query a retriever for product price using its product description.')

# List of all tools
toolz = [tool_search_item, tool_search_image, tool_retrieve_from_text_db]

In [139]:
# Instantiate the agent with error handling
# def _handle_error(error) -> str:
#     return (
#         "The following errors occurred during tool execution:"
#         + error.args[0]
#         + "Please try another tool."
#     )

serp_agent = create_react_agent(llm, toolz, test_prompt_template)

serp_agent_two = create_react_agent(llm_two, toolz, test_prompt_template)

serp_agent_executor = AgentExecutor(agent=serp_agent,
                                    tools=toolz,
                                    verbose=True,
                                    return_intermediate_steps=True,
                                    handle_parsing_errors=True,
                                    callbacks=[error_handler])

serp_agent_executor_two = AgentExecutor(agent=serp_agent_two,
                                    tools=toolz,
                                    verbose=True,
                                    return_intermediate_steps=True,
                                    handle_parsing_errors=True,
                                    callbacks=[error_handler])

In [131]:
description = "Ultra Wireless Noise Cancelling Headphones with Spatial Audio, Over-the-Ear Headphones with Mic, Up to 24 Hours of Battery Life"
image_url = "https://m.media-amazon.com/images/I/81nq8H2IlyL._AC_UL320_.jpg"

In [132]:
# Run the agent executor
serp_agent_executor.invoke({"input": f"what is the estimated, accurate and compact price range of a/an {description}"})



> Entering new AgentExecutor chain...
Action: search text db
Action Input: Ultra Wireless Noise Cancelling Headphones with Spatial Audio, Over-the-Ear Headphones with Mic, Up to 24 Hours of Battery Lifetitle: Location-Enabled Smart Luggage, Backpack, or Property Tags - Unique Medium-Sized Smart ID Tags for Luggage, Carry-on, Backpacks, or Any Other Property Item (Pack of 2)
price: 19.95

title: Passport Holder with Bluetooth Tracker, Works with Apple Find My (iOS Only), Worldwide Locate Passport Cover, Sound & LED Lights Indicator, 3-Year Long Battery Life, Travel Essentials
price: 43.99

title: Passport Holder with Bluetooth Tracker, Works with Apple Find My (iOS Only), Worldwide Locate Passport Cover, Sound & LED Lights Indicator, 3-Year Long Battery Life, Travel Essentials
price: 43.99

title: Luggage Straps, Luggage Straps for Suitcases Add a Bag Easy to Travel Belt for Luggage, Bag Bungee for Luggage, Adjustable Size Luggage Belt and Elastic Suitcase Strap
price: 8.69Action: sea

{'input': 'what is the estimated, accurate and compact price range of a/an Ultra Wireless Noise Cancelling Headphones with Spatial Audio, Over-the-Ear Headphones with Mic, Up to 24 Hours of Battery Life',
 'output': '$349.99-429.00',
 'intermediate_steps': [(AgentAction(tool='search text db', tool_input='Ultra Wireless Noise Cancelling Headphones with Spatial Audio, Over-the-Ear Headphones with Mic, Up to 24 Hours of Battery Life', log='Action: search text db\nAction Input: Ultra Wireless Noise Cancelling Headphones with Spatial Audio, Over-the-Ear Headphones with Mic, Up to 24 Hours of Battery Life'),
   'title: Location-Enabled Smart Luggage, Backpack, or Property Tags - Unique Medium-Sized Smart ID Tags for Luggage, Carry-on, Backpacks, or Any Other Property Item (Pack of 2)\nprice: 19.95\n\ntitle: Passport Holder with Bluetooth Tracker, Works with Apple Find My (iOS Only), Worldwide Locate Passport Cover, Sound & LED Lights Indicator, 3-Year Long Battery Life, Travel Essentials\npr

In [163]:
serp_agent_executor_two.invoke({"input": f"what is the estimated, accurate and compact price range of a/an {description}. Return the price range in the format {{'price_range':{{}}}}"})




> Entering new AgentExecutor chain...
I should first check the database for the price range of the product with the given description.
Action: search text db
Action Input: Ultra Wireless Noise Cancelling Headphones with Spatial Audio, Over-the-Ear Headphones with Mic, Up to 24 Hours of Battery Life

Observationtitle: TSA Approved Luggage Locks, Travel Locks Which Also Work Great as Gym Locks, Suitcase, Backpack and more, 1,2,4 & 6 Pack
price: 20.95

title: Luggage Straps, Luggage Straps for Suitcases Add a Bag Easy to Travel Belt for Luggage, Bag Bungee for Luggage, Adjustable Size Luggage Belt and Elastic Suitcase Strap
price: 8.69

title: Luggage Scale, Large Handle, Electronic Luggage Scale, High Precision Carrying, 110lbs/50kg, Hook Rotates 360°, Travel Airport Checked Luggage Baggage, Avoid Extra Charges for Your Over Weight
price: 6.99

title: Luggage Strap for Carry On Bag - Cinch Belts Luggage - Cinch Travel Belt for Luggage - The Travel Belt for Luggage, Luggage Travel Strap

{'input': "what is the estimated, accurate and compact price range of a/an Ultra Wireless Noise Cancelling Headphones with Spatial Audio, Over-the-Ear Headphones with Mic, Up to 24 Hours of Battery Life. Return the price range in the format {'price_range':{}}",
 'output': "{'price_range': {'min': 349.99, 'max': 429.0}}</s>",
 'intermediate_steps': [(AgentAction(tool='search text db', tool_input='Ultra Wireless Noise Cancelling Headphones with Spatial Audio, Over-the-Ear Headphones with Mic, Up to 24 Hours of Battery Life\n\nObservation', log='I should first check the database for the price range of the product with the given description.\nAction: search text db\nAction Input: Ultra Wireless Noise Cancelling Headphones with Spatial Audio, Over-the-Ear Headphones with Mic, Up to 24 Hours of Battery Life\n\nObservation'),
   'title: TSA Approved Luggage Locks, Travel Locks Which Also Work Great as Gym Locks, Suitcase, Backpack and more, 1,2,4 & 6 Pack\nprice: 20.95\n\ntitle: Luggage Strap

# Test
To test, load your data for use in testing then run

In [30]:
test = pd.read_csv("test.csv")
train = pd.read_csv("train.csv")

In [31]:
# Get the data seen and unseen
train_sample = train.sample(5)
test_sample = test.sample(5)

validation_data = pd.concat([train_sample, test_sample], axis=0)

validation_descriptions = validation_data['title'].values
validation_prices = validation_data['price'].values

In [59]:
# Validate :
results = []
for item_description in validation_descriptions:
    result = serp_agent_executor.invoke({"input": f"what is a reasonable price range of this {item_description}"})
    results.append(result["output"])



> Entering new AgentExecutor chain...
Thought 1: I need to search for the ECOFANTASY Montessori Toddler Busy Book to identify their price range.
Action 1: search internet 
Action Input 1: ECOFANTASY Montessori Toddler Busy Book - New 32 Themes Montessori Toys & Dry Erase Board for Toddlers - My Preschool Kindergarten Learning Activities - Quiet Activity Book for Kids Ages 3 4 5 6 7 8['... 32 Themes Montessori Toys & Dry Erase Board for Toddlers - My Preschool Kindergarten Learning Activities - Quiet Activity Book for Kids Ages 3 4 5 6 7 8 : Toys', 'ECOFANTASY Montessori Toddler Busy Book - New 32 Themes Montessori Toys & Dry Erase Board for ... Learning Activities - Quiet Activity Book for Kids Ages 3 4 5 6 ...', '【MY PRESCHOOL BUSY BOOK】This preschool workbook covers 16 themes, including numbers, alphabet, food & drink, fruits, animals, colors, shapes and more. Each ...', '【32 Comprehensive Theme Busy Book】The Montessori Busy Book covers 16 themes including alphabet, colors, numbers

In [60]:
for result, price in zip(results, validation_prices):
    print(f"The actual price is {price} and the predicted price range is {result}")

The actual price is 13.98 and the predicted price range is $15 - $25
The actual price is 23.0 and the predicted price range is $13.99 - $23.0
The actual price is 19.99 and the predicted price range is $16.99 - $22.99
The actual price is 11.99 and the predicted price range is $2.24 - $8.99
The actual price is 12.99 and the predicted price range is $4 - $8.49
The actual price is 8.99 and the predicted price range is $15 - $20
The actual price is 28.99 and the predicted price range is $3.75 - $5.50
The actual price is 54.97 and the predicted price range is $26.99 - $57.98
The actual price is 35.0 and the predicted price range is $20 - $25
The actual price is 19.47 and the predicted price range is $12 - $25


# Using Image as well through summary

In [149]:
import PIL
import google.generativeai as genai

In [150]:
genai.configure(api_key=os.environ['GOOGLE_API_KEY'])

In [168]:
image = PIL.Image.open('Mcqueen.jpg') 
vision_model = genai.GenerativeModel("gemini-pro-vision", generation_config={"temperature":0.3})


In [164]:
response = vision_model.generate_content([f"Given the description of the product as {description}, extract details about the product in the image such as brand and color and your thought on quality etc. Then generate a searcheable prompt about the product that I can feed to google to get this exact product. Return ONLY the searchable prompt in the format{{'search_prompt':{{}}}} ",image])

In [165]:
response.text

' The product is a pair of over-the-ear headphones that are wireless and have noise-canceling capabilities. They are made by Bose and are available in a cream color. The headphones have a built-in microphone and can be used for up to 24 hours on a single charge.\n\nThe headphones are made of high-quality materials and are very comfortable to wear. They have a sleek design and are very stylish. The sound quality is excellent and the noise-canceling feature works very well. The headphones are also very easy to use.\n\nOverall, I am very impressed with the Bose Ultra Wireless Noise Cancelling Headphones. They are a great pair of headphones for anyone who is looking for a high-quality, wireless, noise-canceling pair of headphones.\n\nHere is a searchable prompt that you can use to find the product on Google:\n\n{bose ultra wireless noise cancelling headphones}'

In [171]:
response = vision_model.generate_content([f"Given me an adequate description of the product in the image in ten words ",image])

In [173]:
d = response.text

In [176]:
serp_agent_executor.invoke({"input": f"what is the estimated, accurate and compact price range of a/an {d}"})




> Entering new AgentExecutor chain...
Action: search text db
Action Input: White leather sneaker with black heel and black laces.title: Unisex Bag Cotton Crew 6-Pair Pack Black/White LG (Men's Shoe 8-12, Women's Shoe 10-13)
price: 27.29

title: Golf Shoe Bag for Men and Women - Mens Womens Shoe Tote
price: 11.99

title: 2 Pairs of 2 Shoe Covers, Black/Gray, 8.25 X 12.75
price: 18.11

title: Double Compartment Shoe Bag, Black, One Size
price: 85.05Action: search internet
Action Input: White leather sneaker with black heel and black laces.[{'position': 1, 'block_position': 'right', 'title': "Thousand Fell | Women's Vegan Leather Lace Up Sneakers | Black", 'price': '$145.00', 'extracted_price': 145.0, 'link': 'https://www.thousandfell.com/products/womens-lace-up-sneaker-black?variant=32185840926807&currency=USD&utm_medium=product_sync&utm_source=google&utm_content=sag_organic&utm_campaign=sag_organic', 'source': 'Thousand Fell', 'rating': 4.7, 'reviews': 314, 'thumbnail': 'https://serpa

{'input': 'what is the estimated, accurate and compact price range of a/an  White leather sneaker with black heel and black laces.',
 'output': '$125-195',
 'intermediate_steps': [(AgentAction(tool='search text db', tool_input='White leather sneaker with black heel and black laces.', log='Action: search text db\nAction Input: White leather sneaker with black heel and black laces.'),
   "title: Unisex Bag Cotton Crew 6-Pair Pack Black/White LG (Men's Shoe 8-12, Women's Shoe 10-13)\nprice: 27.29\n\ntitle: Golf Shoe Bag for Men and Women - Mens Womens Shoe Tote\nprice: 11.99\n\ntitle: 2 Pairs of 2 Shoe Covers, Black/Gray, 8.25 X 12.75\nprice: 18.11\n\ntitle: Double Compartment Shoe Bag, Black, One Size\nprice: 85.05"),
  (AgentAction(tool='search internet', tool_input='White leather sneaker with black heel and black laces.', log='Action: search internet\nAction Input: White leather sneaker with black heel and black laces.'),
   [{'position': 1,
     'block_position': 'right',
     'title'